# Iris data set - Keras
This is an example on machine learning prediction using Iris data set. We will create model to predict species of Iris flower by their sepal width, sepal length, petal width, and petal length.

## Data set
[Iris data set](http://mlr.cs.umass.edu/ml/machine-learning-databases/iris/iris.data) is in the csv format.

### Read set and process
We gonna use [pandas](http://pandas.pydata.org/) python library to process raw data.

In [1]:
import pandas as pd

iris_data = pd.read_csv('data/iris.csv')
iris_data.head()

,sepal_length,sepal_width,petal_length,petal_width,type
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Pandas library has many useful features, for example, ability to query specific data from data set

In [2]:
iris_data.loc[(iris_data['type'] == 
               'Iris-versicolor') & (iris_data['sepal_length'] < 5.0)]

,sepal_length,sepal_width,petal_length,petal_width,type
57,4.9,2.4,3.3,1.0,Iris-versicolor


Here we extract only data we want to use as input

In [3]:
all_inputs = iris_data[['sepal_length', 'sepal_width',
                             'petal_length', 'petal_width']].values

Similarly, we can extract the types of Iris flower, which we use as output data.

In [4]:
all_types = iris_data['type'].values

We make sure that we don't mix up the order of the entries in all_inputs[57] where inputs should correspond to the type in all_types[57] array.

In [16]:
print(all_inputs[57]," - ", all_types[57])

[ 4.9  2.4  3.3  1. ]  -  Iris-versicolor


We convert all the Iris flower types from data set into respective numbers using [numpy](http://www.numpy.org/) library

In [6]:
import numpy as np
outputs_vals, outputs_ints = np.unique(all_types, return_inverse=True)

Now we split data set into training and testing sets

In [7]:
inds = np.random.permutation(len(all_inputs))
train_inds, test_inds = np.array_split(inds, 2)

Also we convert output strings (Iris types) into integers

In [8]:
import keras as kr
outputs_vals, outputs_ints = np.unique(all_types, return_inverse=True)
outputs_cats = kr.utils.to_categorical(outputs_ints)

Using TensorFlow backend.


And now we populate the training and testing arrays

In [9]:
inputs_train, outputs_train = all_inputs[train_inds], outputs_cats[train_inds]
inputs_test,  outputs_test  = all_inputs[test_inds],  outputs_cats[test_inds]

## Machine learning
We will use [keras](https://keras.io/) to create machine learning model and consequential tests.  
Let's start by creating the neural network model.  
 - We create sequential model with input layer which takes 4 inputs  
 - This layer consist of 16 neurons to process the 4 inputs
 - This layer is using sigmoid function to fit outputs in between -1 and 1.
 - And finally we are using softmax function to highlight the one dominant output out of all

In [10]:
model = kr.models.Sequential()
model.add(kr.layers.Dense(16, input_shape=(4,)))
model.add(kr.layers.Activation("sigmoid"))
model.add(kr.layers.Dense(3))
model.add(kr.layers.Activation("softmax"))

### Model configuration
We use adam optimizer and categorical cross entropy as the loss function

In [11]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

### Model training
Model is trained using training data. Number of epochs defines number of training permutations neural network goes through. Batch size determines number of samples per gradient update.

In [12]:
model.fit(inputs_train, outputs_train, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
75/75 [==============================] - 1s 8ms/step - loss: 1.4700 - acc: 0.3200
Epoch 2/100
75/75 [==============================] - 0s 2ms/step - loss: 1.2603 - acc: 0.2400- ETA: 0s - loss: 1.3292 - acc: 0.4000  
Epoch 3/100
75/75 [==============================] - 0s 2ms/step - loss: 1.1535 - acc: 0.3600
Epoch 4/100
75/75 [==============================] - 0s 2ms/step - loss: 1.1033 - acc: 0.3867
Epoch 5/100
75/75 [==============================] - 0s 2ms/step - loss: 1.0721 - acc: 0.4400
Epoch 6/100
75/75 [==============================] - 0s 2ms/step - loss: 1.0467 - acc: 0.5067
Epoch 7/100
75/75 [==============================] - 0s 2ms/step - loss: 1.0227 - acc: 0.6000
Epoch 8/100
75/75 [==============================] - 0s 2ms/step - loss: 0.9911 - acc: 0.6667
Epoch 9/100
75/75 [==============================] - 0s 2ms/step - loss: 0.9520 - acc: 0.7467
Epoch 10/100
75/75 [==============================] - 0s 2ms/step - loss: 0.9087 - acc: 0.6667
Epoch 11/100
75/75 

### Model evaluation
Here we run evaluation on the model we just trained with test data.

In [13]:
loss, accuracy = model.evaluate(inputs_test, outputs_test, verbose=1)

print("\n\nLoss: %10.5f\tAccuracy: %2.2f %%" % (loss, accuracy*100))

75/75 [==============================] - 0s 284us/step


Loss:    0.12991	Accuracy: 100.00 %


### Model test
And finally we test the model by picking one flower and comparing prediction to actual flower type.

In [17]:
prediction = np.around(model.predict(np.expand_dims(inputs_test[57], axis=0))).astype(np.int)[0]
print("Model prediction in hot vector output: %s" % prediction)
print("Model prediction in actual Iris type: %s" % outputs_vals[prediction.astype(np.bool)][0])
print("Actual Iris flower type in data set: Vector: %s Type: %s" % (outputs_test[57].astype(np.int), outputs_vals[prediction.astype(np.bool)][0]))

Model prediction in hot vector output: [0 1 0]
Model prediction in actual Iris type: Iris-versicolor
Actual Iris flower type in data set: Vector: [0 1 0] Type: Iris-versicolor


## Conclusion
Prediction model is behaving as expected and gives us correct prediction with very high accuracy. This model can be saved to be used for futher predictions later.

In [15]:
model.save("iris_model.h5")

Adapted from:
 - `https://github.com/salmanahmad4u/keras-iris/blob/master/iris_nn.py`  
 - `https://github.com/emerging-technologies/keras-iris`